# Problem 1
## a. Using variable elimination, calculate the probability that a student who did well on the exam understands the material.P(+u|+e). Show your work.

P(+e) = 

# Problem 2

In [168]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.linear_model import LinearRegression
import statistics
data = pd.read_csv("Life Expectancy Data.csv")
print(data.axes)
data.describe()

[RangeIndex(start=0, stop=2938, step=1), Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')]


,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,2938.000000,2928.000000,2928.000000,2938.000000,2744.000000,2938.000000,2385.000000,2938.000000,2904.000000,2938.000000,2919.000000,2712.00000,2919.000000,2938.000000,2490.000000,2.286000e+03,2904.000000,2904.000000,2771.000000,2775.000000
mean,2007.518720,69.224932,164.796448,30.303948,4.602861,738.251295,80.940461,2419.592240,38.321247,42.035739,82.550188,5.93819,82.324084,1.742103,7483.158469,1.275338e+07,4.839704,4.870317,0.627551,11.992793
std,4.613841,9.523867,124.292079,117.926501,4.052413,1987.914858,25.070016,11467.272489,20.044034,160.445548,23.428046,2.49832,23.716912,5.077785,14270.169342,6.101210e+07,4.420195,4.508882,0.210904,3.358920
min,2000.000000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.37000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2004.000000,63.100000,74.000000,0.000000,0.877500,4.685343,77.000000,0.000000,19.300000,0.000000,78.000000,4.26000,78.000000,0.100000,463.935626,1.957932e+05,1.600000,1.500000,0.493000,10.100000
50%,2008.000000,72.100000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.75500,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000
75%,2012.000000,75.700000,228.000000,22.000000,7.702500,441.534144,97.000000,360.250000,56.200000,28.000000,97.000000,7.49250,97.000000,0.800000,5910.806335,7.420359e+06,7.200000,7.200000,0.779000,14.300000
max,2015.000000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.60000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000


# Divide the data into a training set and a testing set
I am dividing the training set and testing set by selecting 25% of the data for training (with replacement) and 75% as the test data set.


In [169]:
train = data.sample(frac=0.25)
test = data.sample(frac=0.75)

# Using your training set, determine which variables are actually affecting the life expectancy.How did you arrive at that conclusion?

In [170]:
# To find correlation of each variable perform linear regression in relation to each variable
def print_col_correlations(data):
    filtered = data.values[:,:]
    filtered=filtered[:,4:]
    i = 4
    r_squared_data = []
    for col in filtered.T:
        col = col.astype(float)
        col = np.array(col)

        y =  data.values[:,3]

        y = y.astype(float)
        nans = np.isnan(y)
        y = y[~nans]

        col = col[~nans]

        col_nans = np.isnan(col)
        col=col[~col_nans]
        y = y[~col_nans]
        reg = LinearRegression().fit(col.reshape(-1,1), y)
        #print("score for: "+str(data.columns[i]))
        score = reg.score(col.reshape(-1,1), y)
        r_squared_data.append([data.columns[i],score])
        i+=1
    return pd.DataFrame(
        r_squared_data,columns=["measure_name","r squared"])
print_col_correlations(train)


,measure_name,r squared
0,Adult Mortality,0.427164
1,infant deaths,0.073596
2,Alcohol,0.166320
3,percentage expenditure,0.165231
4,Hepatitis B,0.043076
5,Measles,0.014905
6,BMI,0.312709
7,under-five deaths,0.098453
8,Polio,0.244826
9,Total expenditure,0.039195


Infant mortality ,Hepatitis B, Measles, under-five deaths, Total expenditure and Population	have low correlation with life expectancy. Therefore they will be removed from the analysis.

In [171]:
def drop_rows(data):
    data = data.dropna()
    data.drop('infant deaths',1)
    data.drop('Hepatitis B',1)
    data.drop('Measles ',1)
    data.drop('under-five deaths ',1)
    data.drop('Total expenditure',1)
    data.drop('Population',1)
    return data
def make_model(data):
    
    data = drop_rows(data)
    life_expect = data.values[:,3]
    #print(life_expect)
    reg = LinearRegression().fit(data.values[:,4:], life_expect)
    score = reg.score(data.values[:,4:], life_expect)
    return reg

make_model(train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Evaluate how well does your model predict life expectancy


In [172]:
model = make_model(train)
# dropping rows from test dataset
test_cleaned = drop_rows(test)
x = test_cleaned.values[:,4:]
y=test_cleaned.values[:,3]
print(model.score(x,y))
print(model.coef_)
print(test_cleaned.axes)
#for i in model.coef_:
#    print(i)
#for i in test_cleaned.axes[0]:
#    print(i)
data = []
for coef,axes in zip(model.coef_,test_cleaned.axes[1]):
    data.append([axes,coef])
frame = pd.DataFrame(data,columns=["Axis Name","Coefficient"])
display(frame)

0.8271513109680504
[-1.31208662e-02  1.31341271e-01 -7.74551022e-02  3.79705542e-04
 -7.36629249e-03 -1.20736319e-05  3.67429070e-02 -9.87562977e-02
  1.87833057e-02  1.35195826e-01  1.91663855e-02 -4.76182612e-01
  1.93325107e-05  2.37435340e-10 -1.73413826e-02 -7.63554679e-02
  9.71537168e+00  8.15423579e-01]
[Int64Index([ 984, 1011, 2910, 1193, 1724, 1678,  325,  150, 2929, 2576,
            ...
            1901, 1276,  992, 1630, 1772,    1, 1111, 1338, 2708, 1205],
           dtype='int64', length=1231), Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')]


,Axis Name,Coefficient
0,Country,-1.312087e-02
1,Year,1.313413e-01
2,Status,-7.745510e-02
3,Life expectancy,3.797055e-04
4,Adult Mortality,-7.366292e-03
5,infant deaths,-1.207363e-05
6,Alcohol,3.674291e-02
7,percentage expenditure,-9.875630e-02
8,Hepatitis B,1.878331e-02
9,Measles,1.351958e-01


In [181]:
# determining country
countries = {}
for row in test_cleaned.values:
    #print(row[0])
    #print(row[4:])
    prediction = model.predict(row[4:].reshape(1, -1))[0]
    actual = row[3]
    err = prediction-actual
    #print(repr(err))
    try:
        countries[row[0]].append(err)
    except:
        countries[row[0]]=[err]
#print(countries)
country_stdev_arr = []
for country in countries:
    stdev = statistics.stdev(countries[country])
    #print(stdev)
    country_stdev_arr.append([country,stdev])
display(pd.DataFrame(country_stdev_arr,axes=["Country","Error Standard Deviation"]))


TypeError: __init__() got an unexpected keyword argument 'axes'